In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

def setup_driver():
    #устанавливаем драйвера и связь с ними
    options = webdriver.ChromeOptions()#объект настройки поведения браузера
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    #скрываем автоматизацию и уведомление для сайтов
    options.add_argument('--start-maximized')#чтоб не мучаться со смещением при пеерестройке масштаба (динамика может скрыться и тогда придётся делать дополнительные скролы)
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    #не используем навигатор открыто
    
    return driver

def parse(driver, url):
    try:
        print(f"импортируем в код {url}")
        driver.get(url)
        
        #задержка
        time.sleep(3)
        university_data = {
            'URL': url,
            'Name': 'N/A',
            '2026': 'N/A',
            '2025': 'N/A',
            '2024': 'N/A'
        }
        
        #Опять проходимся по divам в заголовок первого ур и тащим имя университета
        try:
            name_selector = '#__next > main > div > div > div > div > div.css-1dwnmlj > div.desktopHeaderRef.css-wxa6cs > div.css-17ek16l > div > div > div.css-axsv8p > div:nth-child(1) > h1'
            name = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, name_selector))
            )
            university_data['Name'] = name.text.strip()
        except (TimeoutException, NoSuchElementException) as e:
            print(f"У {e} НЕТ НАЗВАНИЯ")
        
        #Путь до контейнера с рейтингами в графике
        try:
            chart = '#rankings > div.css-hny5m5 > div > div > div.css-6dgq65 > div:nth-child(2) > div > svg > g:nth-child(4)'
            WebDriverWait(driver, 10).until(#ждём пока не выполниться условие 10 сек
                EC.presence_of_element_located((By.CSS_SELECTOR, chart))#условие
            )
            
            #прокручиваем страницу к элементу с рейтингами
            rank_el = driver.find_element(By.CSS_SELECTOR, chart)
            driver.execute_script('arguments[0].scrollIntoView(true);', rank_el)
            time.sleep(2)
            
            #где храним(словарь)
            found_rankings = {}
            
            #начинаем с конца (последние 10 меток рейтингов, чтобы наверняка)
            for i in range(10, 0, -1):
                label_selector = f"#chart-scatter-3-labels-{i}"
                try:
                    label_el = driver.find_element(By.CSS_SELECTOR, label_selector)
                    tspan_el = label_el.find_elements(By.TAG_NAME, "tspan")
                    
                    if tspan_el:
                        #то, что надо лежит в тегах tspan в тексте, поэтому мёрджим в текст
                        tspan_texts = []
                        for tspan in tspan_el:
                            text = tspan.text.strip()
                            if text:  #проверяем на пустоту, чтоб в eda было проще
                                tspan_texts.append(text)
                        
                        if tspan_texts:
                            #объединяем
                            full_text = ' '.join(tspan_texts)
                            found_rankings[i] = full_text
                    
                except NoSuchElementException:
                    continue
            
            #сортируем от последней по годам
            sorted_rankings = sorted(found_rankings.items(), key=lambda x: x[0], reverse=True)
            #found_rankings - словарь, так что сортим по ключам в обратном порядке и делаем список кортежей
            
            # вставяляем в соответсвующие столбцы
            for idx, (selector_num, ranking_text) in enumerate(sorted_rankings[:3]):
                if idx == 0:  #первая с конца - 2026
                    university_data['2026'] = ranking_text
                elif idx == 1:  #вторая - 2025
                    university_data['2025'] = ranking_text
                elif idx == 2:  #третья - 2024
                    university_data['2024'] = ranking_text
              
        except (TimeoutException, NoSuchElementException) as e:
            print(f"нет рейтингов {e}")
        
        print(f"парсинг промежутка здесь: {university_data}")
        return university_data
        
    except Exception as e:
        print(f"ссылка {url} плохая: {e}")
        return {
            'URL': url,
            'Name': 'ERROR',
            '2026': 'ERROR',
            '2025': 'ERROR',
            '2024': 'ERROR'
        }

def main():
    #импортируем главный файл
    input_file = 'the_rankings_2026_full.csv'
    output_file = 'universities_rankings.csv' #сюда бы созранилось, если бы парсинг смог дойти до конца
    
    try:
        #читаем файл с ссылками
        df = pd.read_csv(input_file)
        urls = df['URL'].dropna().unique()#в файле были повторы, поэтому почистим
        
        #драйвер
        driver = setup_driver()
        results = []
        
        #переходим по ссылкам
        for i, url in enumerate(urls, 1):
            print(f"на ссылке {i} из {len(urls)}")
            
            if not isinstance(url, str) or not url.startswith('http'):
                print(f"ошибка, пропускаем {url}")
                continue
            
            try:
                data = parse(driver, url)
                results.append(data)
                
                #грузилось долго, поэтому для фиксации результата (чтоб не выскребать из терминала)
                if i % 5 == 0:
                    temp_df = pd.DataFrame(results)
                    temp_df.to_csv(f'temp_results_{i}.csv', index=False)
                    #index=False - типа без индексов
                    print(f'отработало {i} записей)')
            
                time.sleep(2)
                
            except Exception as e:
                print(f'error {url}: {e}')
                continue
        
        #закрываем драйвер
        driver.quit()
        
        #последний файл - создание
        if results:
            result_df = pd.DataFrame(results)
            
            #порядок в файле
            columns_order = ['URL', 'Name', '2026', '2025', '2024']
            #добавляем, что получилось
            final_columns = [col for col in columns_order if col in result_df.columns]
            
            result_df = result_df[final_columns]
            result_df.to_csv(output_file, index=False, encoding='utf-8-sig')
            
            print(f'последний файл: {output_file}')
        else:
            print("нет записей")
    #ошибки прочие        
    except FileNotFoundError:
        print(f"нет входного файла")
    except Exception as e:
        print(f"error: {e}")

if __name__ == '__main__':
    main()